# Self-Driving Car Engineer Nanodegree


## Project: **Finding Lane Lines on the Road** 
***
In this project, you will use the tools you learned about in the lesson to identify lane lines on the road.  You can develop your pipeline on a series of individual images, and later apply the result to a video stream (really just a series of images). Check out the video clip "raw-lines-example.mp4" (also contained in this repository) to see what the output should look like after using the helper functions below. 

Once you have a result that looks roughly like "raw-lines-example.mp4", you'll need to get creative and try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines.  You can see an example of the result you're going for in the video "P1_example.mp4".  Ultimately, you would like to draw just one line for the left side of the lane, and one for the right.

In addition to implementing code, there is a brief writeup to complete. The writeup should be completed in a separate file, which can be either a markdown file or a pdf document. There is a [write up template](https://github.com/udacity/CarND-LaneLines-P1/blob/master/writeup_template.md) that can be used to guide the writing process. Completing both the code in the Ipython notebook and the writeup template will cover all of the [rubric points](https://review.udacity.com/#!/rubrics/322/view) for this project.

---
Let's have a look at our first image called 'test_images/solidWhiteRight.jpg'.  Run the 2 cells below (hit Shift-Enter or the "play" button above) to display the image.

**Note: If, at any point, you encounter frozen display windows or other confounding issues, you can always start again with a clean slate by going to the "Kernel" menu above and selecting "Restart & Clear Output".**

---

**The tools you have are color selection, region of interest selection, grayscaling, Gaussian smoothing, Canny Edge Detection and Hough Tranform line detection.  You  are also free to explore and try other techniques that were not presented in the lesson.  Your goal is piece together a pipeline to detect the line segments in the image, then average/extrapolate them and draw them onto the image for display (as below).  Once you have a working pipeline, try it out on the video stream below.**

---

<figure>
 <img src="examples/line-segments-example.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above) after detecting line segments using the helper functions below </p> 
 </figcaption>
</figure>
 <p></p> 
<figure>
 <img src="examples/laneLines_thirdPass.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your goal is to connect/average/extrapolate line segments to get output like this</p> 
 </figcaption>
</figure>

**Run the cell below to import some packages.  If you get an `import error` for a package you've already installed, try changing your kernel (select the Kernel menu above --> Change Kernel).  Still have problems?  Try relaunching Jupyter Notebook from the terminal prompt.  Also, consult the forums for more troubleshooting tips.**  

## Import Packages

In [1]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

## Read in an Image

In [2]:
#reading in an image
'''
image = mpimg.imread('test_images/whiteCarLaneSwitch.jpg')

#printing out some stats and plotting
print('This image is:', type(image), 'with dimensions:', image.shape)
plt.imshow(image)
'''
# if you wanted to show a single color channel image called 'gray', for example, call as plt.imshow(gray, cmap='gray')

"\nimage = mpimg.imread('test_images/whiteCarLaneSwitch.jpg')\n\n#printing out some stats and plotting\nprint('This image is:', type(image), 'with dimensions:', image.shape)\nplt.imshow(image)\n"

## Ideas for Lane Detection Pipeline

**Some OpenCV functions (beyond those introduced in the lesson) that might be useful for this project are:**

`cv2.inRange()` for color selection  
`cv2.fillPoly()` for regions selection  
`cv2.line()` to draw lines on an image given endpoints  
`cv2.addWeighted()` to coadd / overlay two images
`cv2.cvtColor()` to grayscale or change color
`cv2.imwrite()` to output images to file  
`cv2.bitwise_and()` to apply a mask to an image

**Check out the OpenCV documentation to learn about these and discover even more awesome functionality!**

## Helper Functions

Below are some helper functions to help get you started. They should look familiar from the lesson!

In [3]:
import math

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)
    

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    #line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    #draw_lines(line_img, lines)
    
    return lines

def hough_draw_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    
    return line_img

def weighted_img(img, initial_img, alpha=0.8, beta=1., gamma=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + γ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, alpha, img,beta, gamma)

## Test Images

Build your pipeline to work on the images in the directory "test_images"  
**You should make sure your pipeline works well on these images before you try the videos.**

In [4]:
import os
os.listdir("test_images/")

['whiteCarLaneSwitch.jpg',
 'Thumbs.db',
 'solidWhiteCurve.jpg',
 'solidYellowCurve2.jpg',
 'solidYellowCurve.jpg',
 'C.jpg',
 'solidWhiteRight.jpg']

## Build a Lane Finding Pipeline



Build the pipeline and run your solution on all test_images. Make copies into the `test_images_output` directory, and you can use the images in your writeup report.

Try tuning the various parameters, especially the low and high Canny thresholds as well as the Hough lines parameters.

## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

**Note: if you get an import error when you run the next cell, try changing your kernel (select the Kernel menu above --> Change Kernel). Still have problems? Try relaunching Jupyter Notebook from the terminal prompt. Also, consult the forums for more troubleshooting tips.**

**If you get an error that looks like this:**
```
NeedDownloadError: Need ffmpeg exe. 
You can download it by calling: 
imageio.plugins.ffmpeg.download()
```
**Follow the instructions in the error message and check out [this forum post](https://discussions.udacity.com/t/project-error-of-test-on-videos/274082) for more troubleshooting tips across operating systems.**

In [110]:
from math import isclose

def lines_to_big_line(aline,lower_y=540, upper_y=325):
        '''
        Take in a line, do some algebra and get the bottom point and the top points to plot
        from https://github.com/naokishibuya/car-finding-lane-lines/blob/master/Finding%20Lane%20Lines%20on%20the%20Road.ipynb
        '''
        m,b = aline
        
        if isclose(m,0.):
            return False
            
        y1 = int(lower_y)
        y2 = upper_y     
        
        x1 = int((y1 - b ) / m )
        x2 = int((y2 - b) / m)
        return [(x1,y1),(x2,y2)]

In [130]:
pastleft  = []
pastright = []
from collections import deque
leftq  = deque(maxlen=5)
rightq = deque(maxlen=5)
def process_image2(image):
    
    image_copy = np.copy(image)
    
    '''
    GRAY -> Gaussian blur -> Canny Edge Detection
    (In that order)    
    '''
    gray = grayscale(image_copy)    
    blurred_gray = gaussian_blur(gray,7)    
    canny_low_threshold = 50
    canny_high_threshold = 150
    edges = canny(blurred_gray,canny_low_threshold, canny_high_threshold)
    
    '''
    MASK:
    Next we create the region of interest    
    '''
    mask = np.zeros_like(edges)
    ignore_mask_color = 255
    imshape = image_copy.shape
    
    xl,yl = (475, 315)
    xr,yr = (490, 315)
    vertices = np.array([[(0+50,imshape[0]),(xl,yl), (xr,yr), (imshape[1]-50,imshape[0])]], dtype=np.int32)
    lanes_of_image = region_of_interest(edges, vertices)
    
    '''
    Hough Transform - lines addition    
    '''
    rho = 1
    theta = np.pi/180
    threshold = 10
    min_line_len = 35
    max_line_gap = 18
    #black and red lines from the Houngh Transform 
    lines = hough_lines(lanes_of_image, rho, theta, threshold, min_line_len, max_line_gap)    
    
    left_lines  = []
    right_lines = []
    
    for line in lines:
        
        for x1,y1,x2,y2 in line:
            # y-coord increases and x going to the right so right has positive slope (world upside down and turned)
            slope = (y2-y1)/(x2-x1)
            y_int = y1 - slope*x1
            if slope < 0:
                
                left_lines.append( (slope, y_int) )
                '''left_line_slope  = sum([i[0] for i in left_lines] )/(.0001 + len(left_lines) )                
                left_line_y_int  = sum([i[1] for i in left_lines] )/(.0001 + len(left_lines) )                
                print("got a left line", left_line_y_int)'''
                
            else:
                right_lines.append( ( slope, y_int ) )
    if len(left_lines)==0:
        with open("asdfasdf.frog",'w') as f:
            f.write("left_lines has nothing in it..")
    left_line_slope  = sum([i[0] for i in left_lines] )/(.0001 + len(left_lines) )
    right_line_slope = sum([i[0] for i in right_lines])/(.0001 + len(right_lines) )
    
    left_line_y_int  = sum([i[1] for i in left_lines] )/(.0001 + len(left_lines) )
    right_line_y_int = sum([i[1] for i in right_lines])/(.0001 + len(right_lines) )
    
    left_lines  = (left_line_slope , left_line_y_int )
    right_lines = (right_line_slope, right_line_y_int)    
    
    global pastleft
    global pastright
    
    theleft  = lines_to_big_line(left_lines,  image_copy.shape[0], upper_y=325)
    if not theleft:
        theleft = pastleft
    else:
        pastleft = theleft
        
        
    theright = lines_to_big_line(right_lines, image_copy.shape[0], upper_y=325)
    
    if not theright:
        theright = pastright
    else:
        pastright = theright
    global leftq
    global rightq
    
    leftq.append(theleft)
    rightq.append(theright)
    
    #a.mean(axis=1)
    theleft = np.array(leftq).mean(axis=1)
    theright = np.array(rightq).mean(axis=1)
    
    theleft = ((int(theleft[0][0]), int(theleft[0][1])), (int(theleft[1][0]), int(theleft[1][1])))
    theright = ((int(theright[0][0]), int(theright[0][1])), (int(theright[1][0]), int(theright[1][1])))
    
    image_copy2 = cv2.line(image_copy,theleft[0],theleft[1],  (0,255,0),2)
    image_copy3 = cv2.line(image_copy2,theright[0],theright[1],(0,255,0),2)
    return image_copy3

SyntaxError: invalid syntax (<ipython-input-130-c7a69e293b6d>, line 101)

In [129]:
img = cv2.cvtColor(cv2.imread("./test_images/solidWhiteRight.jpg"),cv2.COLOR_BGR2RGB)
the_lines = process_image2(img)

plt.imshow(the_lines)

NameError: name 'a' is not defined

Let's try the one with the solid white lane on the right first ...

In [124]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML

white_output = 'results2.mp4'
clip1 = VideoFileClip("./test_videos/solidYellowLeft.mp4")

white_clip = clip1.fl_image(process_image2)
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video results2.mp4
[MoviePy] Writing video results2.mp4










  0%|          | 0/682 [00:00<?, ?it/s]







  1%|          | 5/682 [00:00<00:14, 47.85it/s]







  2%|▏         | 16/682 [00:00<00:08, 76.40it/s]







  4%|▍         | 27/682 [00:00<00:07, 87.13it/s]







  6%|▌         | 39/682 [00:00<00:06, 94.09it/s]







  7%|▋         | 47/682 [00:00<00:07, 87.29it/s]







  8%|▊         | 55/682 [00:00<00:07, 83.19it/s]







  9%|▉         | 63/682 [00:00<00:07, 79.03it/s]







 10%|█         | 70/682 [00:00<00:07, 76.52it/s]







 11%|█▏        | 77/682 [00:01<00:08, 73.97it/s]







 12%|█▏        | 84/682 [00:01<00:08, 72.39it/s]







 13%|█▎        | 91/682 [00:01<00:08, 69.77it/s]







 14%|█▍        | 97/682 [00:01<00:08, 68.88it/s]







 15%|█▌        | 103/682 [00:01<00:08, 67.92it/s]







 16%|█▌        | 109/682 [00:01<00:08, 67.20it/s]







 17%|█▋        | 115/682 [00:01<00:08, 66.17it/s]







 18%|█▊        | 121/682 [00:01<00:08, 65.06it/s]







 19%|█▊        | 127/682 [00:01<00:08, 64.30it

 91%|█████████ | 620/682 [00:16<00:01, 38.18it/s]







 91%|█████████▏| 623/682 [00:16<00:01, 37.97it/s]







 92%|█████████▏| 626/682 [00:16<00:01, 37.63it/s]







 92%|█████████▏| 629/682 [00:16<00:01, 37.57it/s]







 93%|█████████▎| 635/682 [00:16<00:01, 37.67it/s]







 94%|█████████▍| 641/682 [00:16<00:01, 37.80it/s]







 95%|█████████▌| 648/682 [00:17<00:00, 37.96it/s]







 96%|█████████▌| 656/682 [00:17<00:00, 38.18it/s]







 97%|█████████▋| 662/682 [00:17<00:00, 38.24it/s]







 98%|█████████▊| 669/682 [00:17<00:00, 38.41it/s]







 99%|█████████▉| 675/682 [00:17<00:00, 38.52it/s]







100%|█████████▉| 681/682 [00:17<00:00, 38.65it/s]

[MoviePy] Done.
[MoviePy] >>>> Video ready: results2.mp4 

CPU times: user 25.2 s, sys: 13.6 s, total: 38.8 s
Wall time: 18.1 s


Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

## Improve the draw_lines() function

**At this point, if you were successful with making the pipeline and tuning parameters, you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform. As mentioned previously, try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines. You can see an example of the result you're going for in the video "P1_example.mp4".**

**Go back and modify your draw_lines function accordingly and try re-running your pipeline. The new output should draw a single, solid line over the left lane line and a single, solid line over the right lane line. The lines should start from the bottom of the image and extend out to the top of the region of interest.**

Now for the one with the solid yellow lane on the left. This one's more tricky!

## Writeup and Submission

If you're satisfied with your video outputs, it's time to make the report writeup in a pdf or markdown file. Once you have this Ipython notebook ready along with the writeup, it's time to submit for review! Here is a [link](https://github.com/udacity/CarND-LaneLines-P1/blob/master/writeup_template.md) to the writeup template file.


## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [10]:
challenge_output = 'test_videos_output/challenge.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip3 = VideoFileClip('test_videos/challenge.mp4').subclip(0,5)
clip3 = VideoFileClip('test_videos/challenge.mp4')
challenge_clip = clip3.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

NameError: name 'process_image' is not defined

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))